In [ ]:
# !export XLA_FLAGS=--xla_gpu_cuda_data_dir=/usr/local/cuda/
!export CUDA_DIR=/usr/lib/cuda/
# !export LD_LIBRARY_PATH=/usr/local/cuda-11.8/lib64:/home/pedro/miniconda3/envs/ml2/lib/
# !export TF_GPU_ALLOCATOR=cuda_malloc_async

In [ ]:
!ls ${CUDA_DIR}/nvvm/libdevice

In [ ]:
# %load_ext autoreload
# %autoreload 2

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras


gpus = tf.config.list_physical_devices("GPU")
print(f"gpus={gpus}")

from tensorflow.keras.datasets import mnist
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Input, Flatten,\
                                    Reshape, LeakyReLU as LR,\
                                    Activation, Dropout
from tensorflow.keras.models import Model, Sequential
from matplotlib import pyplot as plt
from IPython import display # If using IPython, Colab or Jupyter
import numpy as np
import tensorflow_addons as tfa
import datetime
import random
from sklearn.model_selection import train_test_split
import time

import os
import re
import pathlib

In [ ]:
# !wget https://drive.google.com/uc?export=download&id=0B7EVK8r0v71pZjFTYXZWM3FlRnM

## Generate and save random images

In [ ]:
import cv2
from PIL import Image, ImageFilter


data_dir = pathlib.Path("./prepa")
# prepa_dir = pathlib.Path("./prepa")
prepa_files = list(data_dir.glob('*.png'))
all_files = []
for filename in prepa_files:
    all_files.append((filename, 'png'))
for filename in list(data_dir.glob('*.jpg')):
    all_files.append((filename, 'jpg'))
    
def tmp_images_gen():
    for filename, img_type in all_files:
        raw_image = tf.io.read_file(str(filename), name=filename)
        if img_type == "png":
            yield tf.image.decode_png(raw_image, channels=3, name=filename)
        elif img_type == "jpg":
            yield tf.image.decode_jpeg(raw_image, channels=3, name=filename)
            

def unsharp(x):
    image = Image.fromarray(x)
    return image.filter(ImageFilter.UnsharpMask(radius=2, percent=150))

def conservative_smoothing_gray(data, filter_size):
    temp = []
    indexer = filter_size // 2
    new_image = data.copy()
    nrow, ncol, nch = data.shape
    for ch in range(nch):
        for i in range(nrow):
            for j in range(ncol):
                for k in range(i-indexer, i+indexer+1):
                    for m in range(j-indexer, j+indexer+1):
                        if (k > -1) and (k < nrow):
                            if (m > -1) and (m < ncol):
                                temp.append(data[k,m,ch])  
                temp.remove(data[i,j,ch])

                max_value = max(temp)
                min_value = min(temp)

                if data[i,j,ch] > max_value:
                    new_image[i,j,ch] = max_value

                elif data[i,j,ch] < min_value:
                    new_image[i,j,ch] = min_value
                temp =[]
    return new_image.copy()


all_filters = [
    ("Original", lambda x:x),
    ("OpenCV", lambda im: cv2.fastNlMeansDenoisingColored(im,None,10,10,7,21)),
    ("UnsharpMask", unsharp),
    ("OpenCV_UnsharpMask", lambda im: unsharp(cv2.fastNlMeansDenoisingColored(im,None,10,10,7,21))),
    ("bilateralFilter", lambda im: cv2.bilateralFilter(im,9,75,75)),
    ("UnsharpMask_bilateralFilter", lambda im: cv2.bilateralFilter(np.array(unsharp(im)),9,75,75)),
    ("bilateralFilter_UnsharpMask", lambda im: unsharp(cv2.bilateralFilter(im,9,75,75)))

]

# iterr = iter(tmp_images_gen())
# for i in range(8):
#     next(iterr)
# img = next(iterr).numpy()
# # img2 = conservative_smoothing_gray(img,15)
# plt.figure()
# plt.imshow(img)
# plt.figure()
# plt.imshow(img2)
# np.mean(np.abs(img-img2))

In [ ]:
im_iter = iter(tmp_images_gen())
im_iter

In [ ]:
large_image = Image.open('./ds_images/482-Bachillerato-Internacional.jpg')
large_image = large_image.resize((720,480), resample=Image.Resampling.BICUBIC)
# large_image = large_image.crop((80, 150, 200, 280))
# Image.fromarray(np.array(large_image))
large_image = np.array(large_image)

def add_vertical_lines_noise(x, shifts=2, line_height=2):
    x = x.copy()
    axis_for_roll = 1
    y = np.roll(x, -shifts, axis=axis_for_roll)
    j = 0
    for i in range(x.shape[axis_for_roll]):
        if j <= line_height:
            x[:,i,:] = y[:,i,:]
            
        j+=1
        if j == line_height * 2:
            j = 0
#     print(x.shape)
    return x

def add_horizontal_lines_noise(x, shifts=2, line_height=2):
    x = x.copy()
    axis_for_roll = 0
    y = np.roll(x, -shifts, axis=1)
    j = 0
    for i in range(x.shape[axis_for_roll]):
        if j <= line_height:
            x[i,:,:] = y[i,:,:]
            
        
        if j == line_height * 2:
            j = 0
        else:
            j+=1
#     print(x.shape)
    return x

x = add_horizontal_lines_noise(large_image, -7, 1)
plt.figure()
plt.imshow(x)

# im_iter = iter(tmp_images_gen())

# for _ in range(207):
#     next(im_iter)
    
# x = next(im_iter).numpy()
# y = add_horizontal_lines_noise(x, -7, 2)
# # y = add_vertical_lines_noise(x, 10, 2)
# plt.figure(figsize=(15,10))
# plt.subplot(1,2,1)
# plt.imshow(x)
# plt.axis('off')
# plt.subplot(1,2,2)
# plt.imshow(y)
# plt.axis('off')

In [ ]:
im_iter = iter(tmp_images_gen())
im_iter

def create_dir(path):
    isExist = os.path.exists(path)
    if not isExist:
       # Create a new directory because it does not exist
       os.makedirs(path)

def choose_interactive(im_iter, all_filters):

    def render_options(im):
        im = np.array(im)
        plt.figure(figsize=(13,20))
        rows = 4
        cols = 2
        i = 1

        for f_title, f in all_filters:
            plt.subplot(rows,cols,i)
            plt.title(f"{i} {f_title}")
            plt.imshow(f(im))
            plt.axis('off')
            i +=1

        plt.subplots_adjust(wspace = 0.1, hspace = 0.2)

    count = 338
    for c in range(count):
        next(im_iter)

    im = True
    while im is not None:
        im = next(im_iter)
    #     plt.figure()
    #     plt.imshow(im)


        render_options(im)
        plt.ion()
        plt.show()
        x = input()
        count += 1
        display.clear_output()
        print(f"Use {x}, count={count}")
        
        path2 = f"./tmp2/Original/"
        create_dir(path2)
        Image.fromarray(im.numpy()).save(f"{path2}/{count}.jpeg")
        
        for inn in x.split(","):
            if int(inn) == 0:
                continue
            fname, f = all_filters[int(inn)-1]
            result = Image.fromarray(np.array(f(np.array(im))))
            path1 = f"./tmp2/Upgraded/"
            
            create_dir(path1)
            
            result.save(f"{path1}/{count}_{fname}.jpeg")

# choose_interactive(im_iter, all_filters)

In [ ]:
def load_image(filename, ftype):
    raw_image = tf.io.read_file(str(filename), name=filename)
    if ftype == "png":
        return tf.image.decode_png(raw_image, channels=3, name=filename)
    elif ftype == "jpg" or ftype == "jpeg":
        return tf.image.decode_jpeg(raw_image, channels=3, name=filename)
    
original_images = [(re.search('tmp2/Original/([0-9]*)\.jpeg', str(p)).group(1), str(p)) for p in pathlib.Path("./tmp2/Original").glob('*.jpeg')]
upgraded_images = [(re.search('tmp2/Upgraded/([0-9]*)_?([a-zA-Z0-9_]+)\.jpeg', str(p)), str(p)) for p in pathlib.Path("./tmp2/Upgraded").glob('*.jpeg')]

original_images = {a:b for a,b in original_images}
upgraded_images = [(a.group(1), a.group(2), b) for a,b in upgraded_images]

mmap = {}
for indx, op_name, img_path in upgraded_images:
    if indx not in mmap:
        mmap[indx] = [(op_name, img_path)]
    else:
        mmap[indx].append((op_name, img_path))

filter_map = {a:b for a,b in all_filters}

# upgraded_ds = [(original_images[indx], filter_map[op_name], img_path) for indx, op_name, img_path in upgraded_images]


tmap = {a:i for i, (a,b) in enumerate(all_filters)}


def augment(x):
#     seed = (random.randint(0, 100),random.randint(0, 100))
#     x = random_bright(x, training=True)
    x = tf.image.random_brightness(x, 0.2)
#     x = random_contrast(x, training=True)
    x = tf.image.random_contrast(x, 0.2, 0.5)
#     x = random_flip(x, training=True)
#     x = random_rotation(x, training=True)
    x = tf.image.flip_left_right(x)
    x = tf.image.random_flip_up_down(x)
#     x = tf.image.random_hue(x, 0.2)
    return x

def get_chosen_images(train=True):
    original_path = pathlib.Path("./tmp2/Original")
    upgraded_path = pathlib.Path("./tmp2/Upgraded")
    
    
    
    train, test = train_test_split(list(mmap.items()), test_size=0.01, random_state=42)
    print(len(train), len(test))
    if train:
        files = train
    else:
        files = test
    def fn():
        # for indx, filters in mmap.items():
        for indx, filters in files:
            original_filename = original_images[indx]
            original = load_image(original_filename, "jpeg")
            
            original = augment(original)
            
            chosen = original
            for tname, tfn in all_filters:
            
                chosen = tf.concat([chosen, np.asarray(tfn(original.numpy()))], axis=-1)
            
            label = [0.]*7
#             print("CALL")
#             print(filters)
            for tname, upgraded_filename in filters:
#                 print(tname)
#                 print(tmap[tname]-1)
#                 indx, *transf = str(upgraded_filename).split("_")
#                 tname = "_".join(transf).replace(".jpeg","")
                label[tmap[tname]-1] = 1
#                 print(label)
            
#             label = tf.cast(label, tf.float16)
#             print(label, tf.math.reduce_sum(label))
            label = label /tf.math.reduce_sum(label)
            yield chosen, label
    return fn
# iter = get_chosen_images(train=False)()
# next(iter)
# # next(iter)
# img, label = next(iter)
# print(img.shape, label.shape)

# plt.figure()
# plt.imshow(img[:,:,:3])
# plt.figure()
# plt.imshow(img[:,:,3:])

# label

In [ ]:
# class MyConvLayer(tf.keras.layers.Layer):
#   def __init__(self, filters, downscale):
#     super(MyConvLayer, self).__init__()
#     self.filters = filters
#     self.downscale = downscale

#   def build(self, input_shape):
#     reg = tf.keras.regularizers.l1_l2(0.1,0.1)
#     act = layers.LeakyReLU(alpha=0.2)
#     filters = self.filters
#     args = {"padding":'same', "activation":act, "activity_regularizer":reg, "kernel_initializer":tf.keras.initializers.HeNormal(seed=32)}
#     self.conv1 = layers.Conv2D(filters, kernel_size=7, strides=1, **args)
#     self.mp1 = layers.MaxPool2D(self.downscale,self.downscale)
#     self.conv2 = layers.Conv2D(filters, kernel_size=5, strides=1, **args)
#     self.mp2 = layers.MaxPool2D(self.downscale,self.downscale)
#     self.conv3 = layers.Conv2D(filters, kernel_size=3, strides=1, **args)
#     self.mp3 = layers.MaxPool2D(self.downscale,self.downscale)
#     self.conv4 = layers.Conv2D(filters, kernel_size=1, strides=1, **args)

#   def call(self, inputs):
#     x = self.conv1(inputs)
#     x = self.mp1(x)
#     y = self.conv2(inputs)
#     y = self.mp2(y)
#     z = self.conv3(inputs)
#     z = self.mp3(z)
#     return self.conv4(tf.concat([x,y,z], axis=-1))

# def get_categorizer():
#     reg = tf.keras.regularizers.l1_l2(0.1,0.1)
#     act = layers.LeakyReLU(alpha=0.2)
#     #     args = {"padding":'same', "activation":act, "activity_regularizer":reg}
#     args = {"padding":'same', "activation":act, "activity_regularizer":reg, "kernel_initializer":tf.keras.initializers.HeNormal(seed=32)}
#     return tf.keras.Sequential(layers=[
#         tf.keras.layers.Rescaling(1./255),
# #         layers.BatchNormalization(),
#         MyConvLayer(64, 2),
#         layers.MaxPool2D(2,2),
# #         layers.BatchNormalization(),
#         layers.SpatialDropout2D(0.3),
        
#         layers.Conv2D(128, kernel_size=3, strides=1, **args),
#         layers.MaxPool2D(2,2),
# #         layers.BatchNormalization(),
#         layers.SpatialDropout2D(0.3),
        
#         layers.Conv2D(256, kernel_size=3, strides=1, **args),
#         layers.MaxPool2D(2,2),
# #         layers.BatchNormalization(),

        
# # #         layers.SpatialDropout2D(0.3),
# #         layers.Conv2D(256, kernel_size=3, strides=1, **args),
# #         layers.MaxPool2D(2,2),
# #         layers.Conv2D(512, kernel_size=3, strides=1, **args),
# #         layers.MaxPool2D(2,2),
# #         layers.BatchNormalization(),
#         layers.Flatten(),
#         layers.Dense(14, activation='relu', activity_regularizer=reg),
# #         layers.BatchNormalization(),
#         layers.Dense(7, activation='relu', activity_regularizer=reg)
#     ], name="categorizer")
    
# cdata_train = tf.data.Dataset.from_generator(get_chosen_images(train=True), output_signature=(
#         tf.TensorSpec(shape=(480, 720, 24)),
#         tf.TensorSpec(shape=(7)))).batch(1).prefetch(tf.data.AUTOTUNE)
# cdata_test = tf.data.Dataset.from_generator(get_chosen_images(train=False), output_signature=(
#         tf.TensorSpec(shape=(480, 720, 24)),
#         tf.TensorSpec(shape=(7)))).batch(1).prefetch(tf.data.AUTOTUNE)


# initial_learning_rate = 1e-03
# lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
#     initial_learning_rate,
#     decay_steps=100,
#     decay_rate=0.9,
#     staircase=True)


# cmodel = get_categorizer()


In [ ]:
# early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', restore_best_weights=True,
#                                                               patience=10, min_delta=0.)  

# ce = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
# def loss_fn(true_y, pred_y):
# #     print(f"true_y={true_y}, pred_y={pred_y}")
#     return ce(true_y, pred_y)

# cmodel.compile(
#                 optimizer=tfa.optimizers.AdamW(weight_decay=0.00001, learning_rate=lr_schedule),
# #                 optimizer="adam",
#               loss=loss_fn,
#               metrics=['accuracy'])

# # class PrintResults(keras.callbacks.Callback):
# #     def on_epoch_end(self, epoch, logs):
# #         images_path = f"./col_100_output_images/"
# #         image_name = images_path + f"e_{epoch}.jpg"
# # #         display.clear_output()
# #         print_validation(lambda x:cmodel(x, training=False), batch_size=5, save=False, path="./")

# cmodel.fit(cdata_train, validation_data=cdata_test, epochs=100, callbacks=[early_stop])

In [ ]:
# cdata_test = tf.data.Dataset.from_generator(get_chosen_images(train=False), output_signature=(
#         tf.TensorSpec(shape=(480, 720, 24)),
#         tf.TensorSpec(shape=(7)))).batch(1).prefetch(tf.data.AUTOTUNE)

# # ti = next(iter(cdata_test))
# # cmodel.predict()
# ress = []
# i = 0
# for ti in cdata_test:
#     img, y_true = ti
#     img = tf.squeeze(img, axis=0)
#     img = tf.cast(img, tf.uint8)
# #     plt.ion()
# #     plt.subplot(3,1,1)
# #     plt.imshow(img[:,:,:3])
# #     plt.subplot(3,1,2)
# #     plt.imshow(img[:,:,3:6])
# #     plt.subplot(3,1,3)
# #     plt.imshow(img[:,:,6:9])
# #     plt.draw()
# #     plt.pause(0.001)
#     print(y_true, cmodel.predict(ti))
#     i+=1
#     if i == 3:
#         break
        
# # for (img, y_true), y_pred in ress:
# #     print(y_true, y_pred)

In [ ]:
# ress

In [ ]:


def image_generator(pictures = [], large_pictures = [], ds_pictures = [], return_ext=False):
    random.shuffle(pictures)
    def parse_name(filename):
        return "{}_" + re.sub(r"\.(png|jpeg|jpg)", r"_{}.\1", os.path.basename(filename))
    
    def format_name(name, subname):
        return name.format(random.randint(0, 1000), subname)   
    
    def yield_resized(img, fname):
        input_w = 720
        input_h = 480
        cropped = tf.image.resize_with_crop_or_pad(
          img, input_h ,input_w
        )
        variance = tf.math.reduce_variance(tf.cast(cropped, tf.float32))

        if variance > 1000:        
            yield cropped, format_name(fname, "original")
            yield tf.image.flip_left_right(cropped), format_name(fname, "center_crop")
    
    def yield_cropped(img, fname):
        input_w = 720
        input_h = 480
        
        for i in range(200):
            cropped = tf.image.random_crop(
              img, size=[input_h, input_w, 3]
            )

            variance = tf.math.reduce_variance(tf.cast(cropped, tf.float32))
            if variance > 1000:
                yield cropped, format_name(fname, "random_crop")
                yield tf.image.flip_left_right(cropped), format_name(fname, "center_crop")            
    
    def fn():
        input_w = 720
        input_h = 480
        for filename in pictures:
            fname = parse_name(filename)
            raw_png = tf.io.read_file(str(filename), name=filename)
            decoded_png_2 = tf.image.decode_png(raw_png, channels=3, name=filename)
            decoded_png_2 = tf.image.resize(decoded_png_2, [input_h, input_w],
                              method=tf.image.ResizeMethod.BILINEAR)

            yield decoded_png_2, format_name(fname, "original")
            yield tf.image.flip_left_right(decoded_png_2), format_name(fname, "flipped")

        for filename in ds_pictures:
            fname =  parse_name(filename)
            raw_png = tf.io.read_file(str(filename), name=filename)
            decoded_png = tf.image.decode_jpeg(raw_png, channels=3, name=filename)
            
            
            for y in yield_resized(decoded_png, fname):
                yield y

            for y in yield_cropped(decoded_png, fname):
                yield y

#         for filename in large_pictures:
#             fname =  parse_name(filename)
#             raw_png = tf.io.read_file(str(filename), name=filename)
#             decoded_png = tf.image.decode_jpeg(raw_png, channels=3, name=filename)
            
#             for y in yield_resized(decoded_png, fname):
#                 yield y

#             for y in yield_cropped(decoded_png, fname):
#                 yield y
    return fn

In [ ]:
from IPython.display import clear_output
from multiprocessing import Lock, Process, Queue, current_process, Value, Pool, cpu_count
from concurrent import futures

def cache_images():
    data_dir = pathlib.Path("./prepa")
    pictures = list(data_dir.glob('*.png'))

    # TODO: Remove filter
    pictures = random.sample(pictures, 1000)


    data_dir = pathlib.Path("./people")
    large_pictures = list(data_dir.glob('*.jpg'))


    data_dir = pathlib.Path("./ds_images")
    ds_pictures = list(data_dir.glob('*.jpg'))
    
    approx_size = len(pictures)*2 + len(ds_pictures)*2 + len(ds_pictures)*200*2 + len(large_pictures) + len(large_pictures)*20
    
    gen = image_generator(pictures, large_pictures, ds_pictures)
    
    
    print("starting")
    counter = Value('i', 0)
    
        
    it = iter(gen())
    con = True
    
    np = cpu_count()
    print(f'You have {np} cores')



    def f1(im, filename):
            im.numpy()
            
            tf.keras.utils.save_img('./tmp/{}'.format(filename), im)
            with counter.get_lock():
                counter.value += 1

            if (counter.value % 10 == 0):
                clear_output(wait=True)
                print(f"{counter.value+1}/{approx_size}")   
            return f"{im.shape} {filename}"
    with futures.ThreadPoolExecutor(max_workers=min(int((np/3) * 2), 16)) as executor:    
        for im, filename  in it:
            # print(im.shape)
            assert (im.shape[0] == 480 and im.shape[1] == 720)
#             im, filename = next(it)
            future = executor.submit(f1, im, filename)
#             future.add_done_callback(lambda x: print(f"donee: {x}"))
        try:
            data = future.result()
        except Exception as exc:
            print('generated an exception: %s' % exc)
        else:
            print('%r page is' % data)
    

# cache_images()

In [ ]:
# im_iter = iter(tmp_images_gen())
# im_iter
# count = 1180

In [ ]:
# for i in range(count):
#     next(im_iter)

In [ ]:

# images, transf = next(iter)

# plt.subplot(1,2,1)
# plt.imshow(images[:,:,0:3])
# plt.subplot(1,2,2)
# plt.imshow(images[:,:,3:])
# print(transf)

## Load the cached images

In [ ]:


def image_generator_cached(size=None):
    data_dir = pathlib.Path("./tmp")
    # prepa_dir = pathlib.Path("./prepa")
    prepa_files = list(data_dir.glob('*.png'))
    random.shuffle(prepa_files)
    all_files = []
    for filename in prepa_files:
        all_files.append((filename, 'png'))
    for filename in list(data_dir.glob('*.jpg')):
        all_files.append((filename, 'jpg'))
        
        
    train, test = train_test_split(all_files, test_size=0.1, random_state=42)
    print(f"Train: {len(train)} Test:{len(test)}")
    
    random.shuffle(all_files)
    if size:
        all_files = all_files[:size]
    
    def fn():
        for filename, img_type in train:
            raw_image = tf.io.read_file(str(filename), name=filename)
            if img_type == "png":
                yield tf.image.decode_png(raw_image, channels=3, name=filename)
            elif img_type == "jpg":
                yield tf.image.decode_jpeg(raw_image, channels=3, name=filename)
                
    def fn_test():
        for filename, img_type in test:
            raw_image = tf.io.read_file(str(filename), name=filename)
            if img_type == "png":
                yield tf.image.decode_png(raw_image, channels=3, name=filename)
            elif img_type == "jpg":
                yield tf.image.decode_jpeg(raw_image, channels=3, name=filename)
            
    return fn, fn_test

In [ ]:
train_gen, test_gen = image_generator_cached()

train_len = 3113
test_len = 346

# for i, filename in train_gen():
#     i = tf.cast(i, tf.float32)
#     variance = tf.math.reduce_variance(i)
#     if variance < 1000:
#         print(filename, tf.math.reduce_variance(i))
    

In [ ]:
import tensorflow_datasets as tfds
from functools import reduce
splits = tfds.even_splits('train', n=200, drop_remainder=True)

In [ ]:
IMG_H = 480
IMG_W = 720
IMG_CHANNELS = 3

In [ ]:
# train_len = 7164

In [ ]:
BATCH_SIZE = 3
EPOCHS=100
steps_per_epoch=int(train_len/BATCH_SIZE)

In [ ]:
from utils import NoiseUtil, ImgUtils, DataLoader, DataManager

           
def add_noise(x,y):
    downsize_image_ratio = random.choice([1/5])
    sh = tf.cast(tf.shape(x), tf.float32)
    
    print(x)
    
    resized_size_h = sh[0]
    resized_size_w = sh[1]
    down = tf.image.resize(
        x,
        [int(resized_size_h * downsize_image_ratio), int(resized_size_w * downsize_image_ratio)],
        preserve_aspect_ratio=True,
        antialias=False,
        name=None)
    
    

    x= tf.image.resize(
        down,
        [resized_size_h, resized_size_w],
        preserve_aspect_ratio=True,
        antialias=False,
        name=None)
        
    return tf.reshape(x, (resized_size_h, resized_size_w, 3)), y
    
    
#     print(x.shape)

    
#     n = NoiseUtil.pixel_noise(x, random.choice([50,60]), 15, downsize_image_ratios=[1/4, 1/6])

#     n = x + 0.2 * tf.random.normal(
#         x.shape[1:],
#         mean=0.0,
#         stddev=1.0,
#         dtype=tf.dtypes.float32,
#     )

#     return n,y

random_bright = tf.keras.layers.RandomBrightness(factor=0.2)
random_contrast = tf.keras.layers.RandomContrast(factor=0.2)
random_flip = tf.keras.layers.RandomFlip()


def augment(x):
#     seed = (random.randint(0, 100),random.randint(0, 100))
    x = random_bright(x, training=True)
    x = random_contrast(x, training=True)
    x = random_flip(x, training=True)
    return x

def get_train_data():
    return tf.data.Dataset.from_generator(train_gen, output_signature=tf.TensorSpec(shape=(480, 720, 3)))
    
def get_test_data():
    return tf.data.Dataset.from_generator(test_gen, output_signature=tf.TensorSpec(shape=(480, 720, 3)))


def get_dist_ds(ds, ds_len):
    c = ds.map(normm).map(expp).map(augment)
    a = c.map(lambda y: (y,1))
    b = c.map(lambda y: (y,0)).map(add_noise)
    return a.concatenate(b).batch(BATCH_SIZE)

def get_gen_ds(ds):
    return ds.map(augment).map(lambda x: (x,x)).map(add_noise).batch(BATCH_SIZE)

# train_ds = get_gen_ds(get_train_data())
# test_ds = get_gen_ds(get_test_data())

In [ ]:
def print_validation(model=lambda x:x, batch_size=5, save=False, path="./", max_size=20, datasets=([], [])):
        rows = batch_size
        cols = 3
        train_ds, test_ds = datasets
        def print_ds(dataset, save=False):
            results = [(model(x),x, y) for x,y in dataset]
            plt.ion()
            plt.show()
            print(rows, cols)
            plt.figure(figsize=((max_size+15) / cols, max_size / rows))
            for x,x_prev, y in results:
#                 x = (x * 127.5) + 127.5
#                 y = (y * 127.5) + 127.5
#                 x_prev = (x_prev * 127.5) + 127.5
#                 x = x * 255.
#                 y = y * 255.
#                 x_prev = x_prev * 255.
                def info(label, img):
                    print(f"{label} Min:{tf.math.reduce_min(img, axis=[0,1,2])} Max:{tf.math.reduce_max(img, axis=[0,1,2])} Mean:{tf.math.reduce_mean(img, axis=[0,1,2])}, Variance:{tf.math.reduce_variance(img, axis=[0,1,2])}")
    
                info("x: ", x_prev)
                info("y_pred: ", x)
                info("y_true: ", y)

                x = tf.cast(x, tf.uint8)
                x_prev = tf.cast(x_prev, tf.uint8)
                y = tf.cast(y, tf.uint8)
#                 x = tf.clip_by_value(x, 0.0, 1.0)
#                 y = tf.clip_by_value(y, 0.0, 1.0)
                assert x.shape == y.shape
    
                for i in range(x.shape[0]):
                    im = x[i,:,:,:]
                    plt.subplot(cols, rows, i+1)
                    plt.imshow(im)
                    plt.title("Denoised")
                    plt.axis('off')
                    
                for i in range(x_prev.shape[0]):
                    im = x_prev[i,:,:,:]
                    plt.subplot(cols, rows, i+x_prev.shape[0]+1)
                    plt.imshow(im)
                    plt.title("With noise")
                    plt.axis('off')


                for i in range(y.shape[0]):
                    im = y[i,:,:,:]
                    plt.subplot(cols, rows, i+x.shape[0]+x_prev.shape[0]+1)
                    plt.imshow(im)
                    plt.title("Original")
                    plt.axis('off')
            plt.subplots_adjust(wspace = 0.1, hspace = 0.5)
            if save:
                plt.savefig(path)
        
            plt.draw()
            plt.pause(0.001)
            
        print_ds(test_ds.map(lambda x,y: (x[0], y[0])).take(batch_size).batch(batch_size), save=save)
        print_ds(train_ds.map(lambda x,y: (x[0], y[0])).take(batch_size).batch(batch_size), save=False)
        
# print_validation(model=lambda x:x, batch_size=2, save=False, path="./", datasets=(train_ds, test_ds))

In [ ]:
# train_ds, next(iter(train_ds))

In [ ]:
# train_ds = get_gen_ds(get_train_data())
# test_ds = get_gen_ds(get_test_data())

In [ ]:
# x,y = next(iter(train_ds))
# keras.losses.MeanSquaredError()(x,y)
# x = (x * 127.5) + 127.5
# y = (y * 127.5) + 127.5
# x = tf.cast(x, tf.uint8)
# y = tf.cast(y, tf.uint8)
# plt.imshow(x[0])
# plt.figure()
# plt.imshow(y[0])

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
import tensorflow.keras.backend as K

def build_perceptual_loss(input_shape=(480, 720, 3)):
    print(input_shape)
    vgg = VGG16(weights="imagenet", include_top=False, input_shape=input_shape)
    vgg.trainable = False ## Not trainable weights


    selected_layers = ['block1_conv1', 'block2_conv2',"block3_conv3" ,'block4_conv3','block5_conv3']
    selected_layer_weights = [1.0, 4.0 , 4.0 , 8.0 , 16.0]


    outputs = [vgg.get_layer(l).output for l in selected_layers]
    prediction_model = Model(vgg.input, outputs)

    def perceptual_loss(input_image , reconstruct_image):
        input_image = tf.keras.applications.vgg16.preprocess_input(input_image)
        reconstruct_image = tf.keras.applications.vgg16.preprocess_input(reconstruct_image)

        h1_list = prediction_model(input_image)
        h2_list = prediction_model(reconstruct_image)

        rc_loss = 0.0

        for h1, h2, weight in zip(h1_list, h2_list, selected_layer_weights):
            h1 = K.batch_flatten(h1)
            h2 = K.batch_flatten(h2)
            rc_loss = rc_loss + weight * K.sum(K.square(h1 - h2), axis=-1)

        return rc_loss
    
    return perceptual_loss

In [ ]:
train_len

small_epochs = 30


small_steps_per_epoch = int((train_len/small_epochs)/BATCH_SIZE)
small_steps_per_epoch

In [ ]:
def build_autoencoder():
    inputs = keras.Input(shape=(480, 720, 3))
    reg = tf.keras.regularizers.l1_l2(0.1,0.1)
    act = layers.LeakyReLU(alpha=0.2)
#     args = {"padding":'same', "activation":act, "activity_regularizer":reg}
    args = {"padding":'same', "activation":act, "kernel_initializer":tf.keras.initializers.HeNormal(seed=32)}
    
    x = tf.keras.layers.Rescaling(1./255)(inputs)
    x = layers.Conv2D(128, kernel_size=5, strides=1, dilation_rate=2, **args)(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPool2D(2,2)(x)
    x = layers.SpatialDropout2D(0.3)(x)
    x = layers.Conv2D(128, kernel_size=3, strides=1, dilation_rate=2, **args)(x)
    x = layers.MaxPool2D(2,2)(x)
    x = layers.SpatialDropout2D(0.3)(x)
    x = layers.Conv2D(128, kernel_size=3, strides=1, **args)(x)
    x = layers.MaxPool2D(2,2)(x)
    x = layers.Conv2D(256, kernel_size=5, strides=2, **args)(x)
    x = layers.Conv2D(512, kernel_size=(2,3), strides=(2,3), **args)(x)
    
    x = layers.Conv2DTranspose(512, kernel_size=(3,3), strides=(2,3), **args)(x)
    x = layers.Conv2DTranspose(256, kernel_size=4, strides=2, **args)(x)
    x = layers.Conv2DTranspose(128, kernel_size=2, strides=2, **args)(x)
    x = layers.Conv2DTranspose(128, kernel_size=3, strides=1, **args)(x)
    x = layers.UpSampling2D(2)(x)
    x = layers.Conv2DTranspose(64, kernel_size=3, strides=1, **args)(x)
    x = layers.UpSampling2D(2)(x)
    x = layers.Conv2DTranspose(32, kernel_size=3, strides=1, **args)(x)
    x = layers.Conv2DTranspose(3, kernel_size=3, strides=1, **args)(x)
    
    return tf.keras.Model(inputs, x)
    

autoencoder = build_autoencoder()

class RenderImages(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        images_path = f"./col_100_output_images/"
        image_name = images_path + f"e_{epoch}.jpg"
#         display.clear_output()
        print_validation(lambda x:autoencoder(x, training=False), batch_size=5, save=False, path="./")
        
class LRMetric(tf.keras.metrics.Mean):
    def update_state(self, y_true, y_pred, sample_weight=None):
        lr = autoencoder.optimizer.lr
        current_lr = lr(autoencoder.optimizer.iterations)
        super().update_state(current_lr)
        
# early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', restore_best_weights=False,
#                                                               patience=10, min_delta=0.)  

       


## Remove horizontal lines

In [ ]:
people = [(str(p),'jpeg') for p in pathlib.Path("./people").glob('*.jpg')]
ds_images = [(str(p),'jpeg') for p in pathlib.Path("./ds_images").glob('*.jpg')]
large_images = [*people, *ds_images]

In [ ]:
random.shuffle(large_images)
large_images

In [ ]:
def pixelation_noise(x):
    downsize_image_ratio = random.choice([1/2,1/5,1/10, 1/12])
    sh = tf.cast(tf.shape(x), tf.float32)
    
    resized_size_h = sh[0]
    resized_size_w = sh[1]
    down = tf.image.resize(
        x,
        [int(resized_size_h * downsize_image_ratio), int(resized_size_w * downsize_image_ratio)],
        preserve_aspect_ratio=True,
        antialias=False,
        name=None)
    x= tf.image.resize(
        down,
        [resized_size_h, resized_size_w],
        preserve_aspect_ratio=True,
        antialias=False,
        name=None)
    return x

def get_right_dims(shape):
    print(shape)
    h, w, c = shape
    # 624, 936
    while h > 624*1.5 and w > 936*1.5:
        h *= 0.9
        w *= 0.9
    h *= 1.1
    w *= 1.1
    return (int(h),int(w))

def random_brightness(x):
#     x = tf.image.random_brightness(x, 0.2)
    x = tf.image.random_contrast(x, 0.2, 0.5)
    return x

def manual_filter(slice_paths):
    count = 1055
    for idx, (filename, img_type) in enumerate(slice_paths):
        input_w = 720
        input_h = 480
        raw_image = tf.io.read_file(filename, name=filename)
        if img_type == "png":
            img = tf.image.decode_png(raw_image, channels=3, name=filename)
        elif img_type == "jpg" or img_type == "jpeg":
            img = tf.image.decode_jpeg(raw_image, channels=3, name=filename)
            
        h,w = get_right_dims(img.shape)

        img = tf.image.resize(
            img,
            [h, w],
            preserve_aspect_ratio=True,
            antialias=False,
            name=None)
        img = tf.cast(img, tf.uint8)

        for i in range(100):
            # * 1.3
            cropped = tf.image.random_crop(
              img, size=[624, 936, 3]
            )
            cropped_small = tf.image.resize(
                cropped,
                [480, 720],
                preserve_aspect_ratio=True,
                antialias=False,
                name=None)
            
            ts = [lambda x: add_horizontal_lines_noise(x.numpy(), random.randint(-10, -8), random.randint(2, 5)),
                 pixelation_noise,
                  lambda x: add_horizontal_lines_noise(pixelation_noise(x).numpy(), random.randint(-10, -8), random.randint(2, 5))
                 ]
            
            fn = random.choice(ts)
            cropped_small = fn(cropped_small)
            
            cropped_small = tf.cast(cropped_small, tf.uint8)
            plt.figure(figsize=(20,10))
            plt.subplot(1,2,1)
            plt.title(f"{cropped.shape}")
            plt.imshow(cropped)
            plt.axis("Off")
            plt.subplot(1,2,2)
            plt.title(f"{cropped_small.shape}")
            plt.imshow(cropped_small)
            plt.axis("Off")
            plt.ion()
            plt.show()
            x = input()
            
            display.clear_output()
            print(f"Count={count}")
            if x == "3":
                break
            if x == "1":
                count += 1
                print("Saved!")
                path1 = "./upscale_imgs/"
                create_dir(path1)
                Image.fromarray(cropped.numpy()).save(f"{path1}/{count}_large.jpeg")
                Image.fromarray(cropped_small.numpy()).save(f"{path1}/{count}_small.jpeg")

# manual_filter(large_images)

#                 for y1 in yield_resized_cropped(img, "p{}_{}.jpeg"):

In [ ]:
upscale_ds = [(re.search('upscale_imgs/([0-9]*)_?([a-zA-Z0-9_]+)\.jpeg', str(p)), str(p), 'jpeg') for p in pathlib.Path("./upscale_imgs").glob('*.jpeg')]
upscale_ds = [(r.group(1), r.group(2), (fname, ftype)) for r, fname, ftype in upscale_ds]
upscale_ds_map = {}
for idx, itype, fdata in upscale_ds:
    if idx not in upscale_ds_map:
        upscale_ds_map[idx] = {}
    upscale_ds_map[idx][itype] = fdata
# upscale_ds.sort()
upscale_ds = [(fdata["small"], fdata["large"]) for fdata in upscale_ds_map.values()]

def gen_img(filename, img_type):
    raw_image = tf.io.read_file(filename, name=filename)
    if img_type == "png":
        return tf.image.decode_png(raw_image, channels=3, name=filename)
    elif img_type == "jpg" or img_type == "jpeg":
        return tf.image.decode_jpeg(raw_image, channels=3, name=filename)

def yield_small_and_large():
    random.shuffle(upscale_ds)
    train, test = train_test_split(upscale_ds, test_size=0.1, random_state=42)
    print(f"Train: {len(train)} Test:{len(test)}")
    
    def train_gen():
        for indx, (small, large) in enumerate(train):
            small_img = gen_img(*small)
            large_img = gen_img(*large)
            large_img = tf.image.resize(large_img, (810, 1215))
            yield tf.random.Generator.from_seed(int(idx)).normal(shape=[2]), small_img, large_img
    
    def test_gen():
        for indx, (small, large) in enumerate(test):
            small_img = gen_img(*small)
            large_img = gen_img(*large)
            large_img = tf.image.resize(large_img, (810, 1215))
            yield tf.random.Generator.from_seed(int(idx)).normal(shape=[2]), small_img, large_img
    return train_gen, test_gen
        
train_gen, test_gen = yield_small_and_large()

def augment_img(x):
#     x = tf.image.stateless_random_brightness(x, 0.2, seed)
#     x = tf.image.random_contrast(x, 0.2, 0.5)
    x = tf.image.random_flip_left_right(x)
    x = tf.image.random_flip_up_down(x)
    return x

def augment(idx, x, y):
    return augment_img(idx, x), augment_img(idx, y)

def create_ds(gen_fn):
    return tf.data.Dataset.from_generator(gen_fn, output_signature=(
                tf.TensorSpec(shape=(2)),
                tf.TensorSpec(shape=(480, 720, 3)),
                tf.TensorSpec(shape=(810, 1215, 3))
        )).map(augment)

large_train, large_test = train_test_split(large_images, test_size=0.1, random_state=42)



def create_ds_2(image_paths):
    def gen_fn_inner():
        for image_name, image_type in image_paths:
            img = gen_img(image_name, image_type)
            
            if img.shape[0] < 720 or img.shape[1] < 1080:
                continue
            
            cache = []
            for _ in range(10):
            
                large_img = tf.image.random_crop(
                  img, size=[720, 1080, 3]
                )
                
                large_img = augment_img(large_img)
            
                small_img = tf.image.resize(large_img, (480, 720))

                ts = [lambda x: add_horizontal_lines_noise(x.numpy(), random.randint(-10, -8), random.randint(2, 5)),
                     pixelation_noise,
                      lambda x: add_horizontal_lines_noise(pixelation_noise(x).numpy(), random.randint(-10, -8), random.randint(2, 5))
                     ]

                fn = random.choice(ts)
                small_img = fn(small_img)
                small_img = tf.cast(small_img, tf.uint8)

                cache.append((small_img, large_img ))
            
            random.shuffle(cache)
            for c in cache:
                yield c
    return tf.data.Dataset.from_generator(gen_fn_inner, output_signature=(
                tf.TensorSpec(shape=(480, 720, 3)),
                tf.TensorSpec(shape=(720, 1080, 3))
        ))

# i = iter(large_train_ds)
# x,y = next(i)
# x = tf.cast(x, tf.uint8)

# plt.figure()
# plt.imshow(x[0])

In [ ]:
# for i, (i1, i2) in enumerate(create_ds_2(large_train).batch(1)):
#     print(i, i1.shape, i2.shape)

In [ ]:
1023/3

In [ ]:
# large_train_ds = create_ds(train_gen).batch(2)
# large_test_ds = create_ds(test_gen).batch(1)

large_train_ds = create_ds_2(large_train).batch(2)
large_test_ds = create_ds_2(large_test).batch(1)

perceptual_loss = build_perceptual_loss(input_shape=(720, 1080, 3))

#x = tfa.layers.GroupNormalization()(x)
# = layers.SpatialDropout2D(0.3)(x)
#x = layers.UpSampling2D((2,3))(x)

class MyRescale(tf.keras.layers.Layer):
  def __init__(self):
    super(MyRescale, self).__init__()
  def build(self, input_shape):
     self.kernel = self.add_weight("kernel", initializer=tf.keras.initializers.Constant(value=255))


  def call(self, inputs):
   return inputs * self.kernel

class MyConcat(tf.keras.layers.Layer):
  def __init__(self):
    super(MyConcat, self).__init__()
  def build(self, input_shape):
     self.kernel = self.add_weight("kernel", initializer=tf.keras.initializers.Constant(value=1))


  def call(self, inputs):
    x, y = inputs
    return tf.concat([x,y* self.kernel], axis=-1) 

class MyDeconv(tf.keras.layers.Layer):
    def __init__(self, filters, kernel_size, strides, name, resize_to=None, norm=True):
        super(MyDeconv, self).__init__(name=name)
        self.filters = filters
        self.kernel_size = kernel_size
        self.resize_to = resize_to
        self.strides = strides
        self.norm = norm
        
    def get_config(self):
        config = super().get_config()
        config.update({
            "filters": self.filters,
            "kernel_size": self.kernel_size,
            "resize_to": self.resize_to,
            "strides": self.strides,
        })
        return config
        
    def build(self, input_shape):
        filters = self.filters
        kernel_size = self.kernel_size
        name = self.name
        
        strides = self.strides if isinstance(self.strides, tuple) else (self.strides,self.strides)
                     
        if self.resize_to is None:
            h,w = input_shape[1], input_shape[2]
            self.resize_to = (int(h*strides[0]),int(w*strides[1]))
        
        self.resize_layer = layers.Lambda(lambda x: tf.image.resize(x, self.resize_to, method="nearest"), name=f"resize_nearest_{name}")
        
        def build_conv(filters, kernel_size, name):
            conv = layers.Conv2D(filters, kernel_size=kernel_size, strides=1, padding="same", use_bias=True,
                              activation="relu",
    #                           kernel_initializer=tf.keras.initializers.HeNormal(seed=32),
                              name=name)
            if self.norm:
                return tfa.layers.SpectralNormalization(conv, name=f"{name}_spectral_norm")
            else:
                return conv
            
        self.conv1 = build_conv(filters, kernel_size, name=f"{name}_1")
        self.conv2 = build_conv(filters, kernel_size, name=f"{name}_2")

    def call(self, inputs):
        x = self.resize_layer(inputs)
        x = self.conv1(x)
        x = self.conv2(x)
        return x

# def get_activation():
    # return layers.LeakyReLU(alpha=0.2)
act = layers.LeakyReLU(alpha=0.2)

def maxconv_name():
    i = 0
    while True:
        i += 1
        yield f"max_conv_{i}"
        
def deconv_name():
    i = 0
    while True:
        i += 1
        yield f"deconv_{i}"
        
max_conv_name = iter(maxconv_name())
deconv_name_i = iter(deconv_name())

def deconv(filters, kernel_size, strides, name, norm=False, activation="relu", **args):
    if norm:
        return tfa.layers.SpectralNormalization(
            layers.Conv2DTranspose(filters, kernel_size=kernel_size, strides=strides, padding="same", use_bias=False,
                          activation=activation, kernel_initializer=tf.keras.initializers.HeNormal(seed=32),
                          name=name, **args), name=f"{name}_spectral_norm")
    return layers.Conv2DTranspose(filters, kernel_size=kernel_size, strides=strides, padding="same", use_bias=False, 
                      activation=activation, kernel_initializer=tf.keras.initializers.HeNormal(seed=32),
                      name=name, **args)

def conv(filters, kernel_size, strides, name, norm=False, use_init = True, use_bias=False, activation="relu", **args):
    if norm:
        return tfa.layers.SpectralNormalization(
            layers.Conv2D(filters, kernel_size=kernel_size, strides=strides, padding="same", use_bias=use_bias,
                          activation=activation,
                          kernel_initializer=tf.keras.initializers.HeNormal(seed=32) if use_init else None,
                          name=name, **args), name=f"{name}_spectral_norm")
    return layers.Conv2D(filters, kernel_size=kernel_size, strides=strides, padding="same", use_bias=use_bias,
                      activation=activation,
                      kernel_initializer=tf.keras.initializers.HeNormal(seed=32) if use_init else None,
                      name=name, **args)

def maxpoolconv(filters, pool_kernel_size, kernel_size=3, strides=1, norm=True):
    name = next(max_conv_name)
    def fn(x):
        x = conv(filters, kernel_size=kernel_size, strides=1, name=name, norm=norm)(x)
        return layers.MaxPool2D(pool_kernel_size, name=f"{name}_maxpool")(x)
    return fn

# def upscaleconv(filters, pool_kernel_size, kernel_size=3, strides=1, norm=True):
#     name = next(deconv_name_i)
#     def fn(x):
#         x = layers.UpSampling2D(pool_kernel_size, name=f"upsampling_{name}")(x)
#         x = deconv(filters, kernel_size=kernel_size, strides=strides, name=name, norm=norm)(x)
#         return x
#     return fn

# def resizeconv(filters, new_size, kernel_size=3, strides=1, norm=True):
#     name = next(deconv_name_i)
#     def fn(x):
#         x = layers.Lambda(lambda x: tf.image.resize(x, new_size, method="nearest"), name=f"resize_nearest_{name}")(x)
#         x = deconv(filters, kernel_size=kernel_size, strides=strides, name=name, norm=norm)(x)
#         return x
#     return fn

def build_autoencoder_large():
    inputs = keras.Input(shape=(480, 720, 3))
    norm = False
    
    x = tf.keras.layers.Rescaling(1./255)(inputs)
    
    ffs = []
    
    x = conv(16, 3, 1, name="first_conv3", use_init=False, use_bias=True, norm=norm)(x)
    ffs.append(layers.SpatialDropout2D(0.8)(x))
    x = maxpoolconv(32, 2, norm=norm)(x)
    ffs.append(layers.SpatialDropout2D(0.8)(x))
    x = maxpoolconv(64, 2, norm=norm)(x)
    ffs.append(layers.SpatialDropout2D(0.8)(x))
    x = maxpoolconv(128, 2, norm=norm)(x)
    ffs.append(layers.SpatialDropout2D(0.8)(x))
    x = maxpoolconv(256, 2, norm=norm)(x)
    ffs.append(layers.SpatialDropout2D(0.8)(x))
#     x = layers.SpatialDropout2D(0.2)(x)
    x = maxpoolconv(512, (2,3), norm=norm)(x)
    x = layers.SpatialDropout2D(0.2)(x)
    
    ffs.reverse()

    x = MyDeconv(512, 3, (2,3), name="mydeconv7", resize_to=None, norm=norm)(x)
    x = layers.SpatialDropout2D(0.2)(x)
    x = tf.concat([x, ffs[0]], axis=-1)
    x = MyDeconv(256, 3, 2, name="mydeconv6", resize_to=None, norm=norm)(x)
    x = layers.SpatialDropout2D(0.2)(x)
    x = tf.concat([x, ffs[1]], axis=-1)
    x = MyDeconv(128, 3, 2, name="mydeconv5", resize_to=None, norm=norm)(x)
    x = tf.concat([x, ffs[2]], axis=-1)
    x = MyDeconv(64, 3, 2, name="mydeconv4", resize_to=None, norm=norm)(x)
    x = tf.concat([x, ffs[3]], axis=-1)
    x = MyDeconv(32, 3, 2, name="mydeconv3", resize_to=None, norm=norm)(x)
    x = tf.concat([x, ffs[4]], axis=-1)
#     x = MyDeconv(128, 3, 1, name="mydeconv2", resize_to=(360, 540))(x)
    x = MyDeconv(16, 3, 1, name="mydeconv1", resize_to=(720, 1080), norm=norm)(x)
    x = conv(9, 3, 1, name="last_conv3", use_init=False, use_bias=True, norm=norm)(x)
    x = conv(3, 3, 1, name="last_conv1", activation="sigmoid", use_init=False, use_bias=True, norm=norm)(x)

#     x = deconv(64, kernel_size=1, strides=1, name="prelast_deconv", norm=True)(x)
#     x = deconv(3, kernel_size=1, strides=1, name="last_deconv", norm=True)(x)
#     x = layers.Conv2DTranspose(3, kernel_size=3, strides=1, padding="same", 
#                       activation="sigmoid", kernel_initializer=tf.keras.initializers.HeNormal(seed=32))(x)
    
#     x = MyRescale()(x)
    x = tf.keras.layers.Rescaling(255.)(x)
    
    return tf.keras.Model(inputs, x)
    

class RenderImages(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        images_path = f"./col_100_output_images/"
        image_name = images_path + f"e_{epoch}.jpg"
#         display.clear_output()
        print_validation(lambda x:autoencoder(x, training=False), batch_size=5, save=False, path="./")
        
class LRMetric(tf.keras.metrics.Mean):
    def update_state(self, y_true, y_pred, sample_weight=None):
        lr = autoencoder.optimizer.lr
        current_lr = lr(autoencoder.optimizer.iterations)
        super().update_state(current_lr)
        
# early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', restore_best_weights=False,
#                                                               patience=10, min_delta=0.)  

autoencoder = build_autoencoder_large()
autoencoder.summary()

In [ ]:
l1 = 200
l2 = 12

(l1 // l2) * l2

In [ ]:
autoencoder = build_autoencoder_large()

initial_learning_rate = 5e-5
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=500,
    decay_rate=0.95,
    staircase=True)

class RenderImages(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        images_path = f"./col_100_output_images/"
        image_name = images_path + f"e_{epoch}.jpg"
#         display.clear_output()
        print_validation(lambda x:autoencoder(x, training=False), batch_size=2, save=False, path="./", datasets=(large_train_ds, large_test_ds))
        
        
# class RenderImagesWithCast(keras.callbacks.Callback):
#     def on_batch_end(self, epoch, logs):
#         it = iter(large_test_ds.take(1)
#         img = next(it)
#         plt.figure()
#         plt.imshow(img)
#         plt.figure()
#         print(img.shape)
#         y_pred = autoencoder.predict(tf.expand_dims(img, axis=0))
#         y_pred = tf.cast(y_pred, tf.uint8)
#         plt.imshow(y_pred[0])
        
#         print_validation(lambda x:autoencoder(x, training=False), batch_size=2, save=False, path="./", datasets=(large_train_ds, large_test_ds))
        
        
opt = tfa.optimizers.AdamW(weight_decay=0.0001, learning_rate=lr_schedule, beta_1=0.8, beta_2=0.99)

checkpoint_dir = './prepa_autoencoder_ckpt'
checkpoint = tf.train.Checkpoint(model=autoencoder, optimizer=opt)
ckpt_manager = tf.train.CheckpointManager(checkpoint, checkpoint_dir, max_to_keep=5)

# try:
#     if ckpt_manager.latest_checkpoint:
#         checkpoint.restore(ckpt_manager.latest_checkpoint)
# except:
#     print("Could not restore the checkopint")
    
def on_epoch_end(batch, logs):
    ckpt_save_path = ckpt_manager.save()
                                                        
lm = tf.keras.callbacks.LambdaCallback(on_epoch_end=on_epoch_end)

class CovMetric(tf.keras.metrics.Mean):
    def update_state(self, y_true, y_pred, sample_weight=None):
        cov = tf.keras.losses.CategoricalCrossentropy(from_logits=True)(y_true, y_pred)
        super().update_state(cov)
        
        
mse = tf.keras.losses.MeanSquaredError()

def ploss_ce(true_y, pred_y):
    return perceptual_loss(true_y, pred_y)
    # print(f"{label} Min:{tf.math.reduce_min(img, axis=[0,1,2])} Max:{tf.math.reduce_max(img, axis=[0,1,2])} Mean:{tf.math.reduce_mean(img, axis=[0,1,2])}, Variance:{tf.math.reduce_variance(img, axis=[0,1,2])}")
    
#     min_loss = mse(tf.math.reduce_min(true_y, axis=[1,2]), tf.math.reduce_min(pred_y, axis=[1,2]))
#     max_loss = mse(tf.math.reduce_max(true_y, axis=[1,2]), tf.math.reduce_max(pred_y, axis=[1,2]))
# #     mean_loss = mse(tf.math.reduce_mean(true_y, axis=[1,2]), tf.math.reduce_mean(pred_y, axis=[1,2]))
# #     variance_loss = mse(tf.math.reduce_variance(true_y, axis=[1,2]), tf.math.reduce_variance(pred_y, axis=[1,2]))
    
#     return mse(true_y, pred_y) + min_loss + max_loss 
# #     loss = perceptual_loss(true_y, pred_y)
# # #     mse_loss = 0.0
#     mse_loss = mse(true_y, pred_y) + 0.000001
#     mse_loss = ((loss // mse_loss) // 100) * mse_loss
    
#     return loss + mse_loss
        

version = "v9.1_slower"
    
autoencoder.compile(
#                     optimizer="adam",
#                     optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule, beta_1=0.6),
#                     optimizer=tf.keras.optimizers.experimental.SGD(learning_rate=1e-5),
#                 tfa.optimizers.LazyAdam(0.001),
                optimizer=tfa.optimizers.AdamW(weight_decay=0.0001, learning_rate=lr_schedule, 
                                               beta_1=0.8, beta_2=0.99
#                                                , clipnorm=5
                                              ),
#                     loss='mean_squared_error',
                loss=ploss_ce,
                    metrics=["mse"],
#                    metrics=[LRMetric()],
                )

early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', restore_best_weights=False,
                                                              patience=10, min_delta=0.)  

tboard_callback = tf.keras.callbacks.TensorBoard(log_dir = f"./prepa_autencoder_logs/{version}",
                  write_graph=True,
                  histogram_freq = 1,
                  update_freq="epoch"
                  )

ghistory = autoencoder.fit(large_train_ds.prefetch(tf.data.AUTOTUNE),
                                epochs=200, 
#                                     steps_per_epoch=5, 
#                                     steps_per_epoch=small_steps_per_epoch,
                                validation_data=large_test_ds,
      callbacks=[RenderImages(), lm, tboard_callback])

In [ ]:
it = iter(large_train_ds)
for i in range(20):
    next(it)
img, img2 = next(it)



x = img
x = tf.keras.layers.Rescaling(1./255)(x)
x = layers.MaxPool2D(2,2)(x)
plt.imshow(x[0])
plt.figure()
x = layers.MaxPool2D(2,2)(x)
plt.imshow(x[0])
plt.figure()
x = layers.MaxPool2D(2,2)(x)
plt.imshow(x[0])
plt.figure()
x = layers.MaxPool2D(2,2)(x)
plt.imshow(x[0])
plt.figure()
x = layers.MaxPool2D(2,3)(x)
plt.imshow(x[0])
plt.figure()
# x = layers.MaxPool2D(2,2)(x)
# x = layers.MaxPool2D(2,2)(x)
# x = layers.UpSampling2D((2,2))(x)
# 
x = layers.UpSampling2D((2,3))(x)
x = layers.UpSampling2D((2,2))(x)
x = layers.UpSampling2D((2,2))(x)
x = layers.UpSampling2D((2,2))(x)
x = layers.Lambda(lambda x: tf.image.resize(x, (360, 540), method="nearest"))(x)
x = layers.Lambda(lambda x: tf.image.resize(x, (720, 1080), method="nearest"))(x)
x = tf.keras.layers.Rescaling(255)(x)

def info(label, img):
    print(f"{label} Min:{tf.math.reduce_min(img, axis=[0,1,2])} Max:{tf.math.reduce_max(img, axis=[0,1,2])} Mean:{tf.math.reduce_mean(img, axis=[0,1,2])}, Variance:{tf.math.reduce_variance(img, axis=[0,1,2])}")
    
info("IMG", img)
info("X", x)

img = tf.cast(img, tf.uint8)
img2 = tf.cast(img2, tf.uint8)
x = tf.cast(x, tf.uint8)

plt.imshow(img[0])
plt.figure()
plt.imshow(x[0])



In [ ]:
x,y = 480, 720

x,y = x/2,y/2
print(x,y)
x,y = x/2,y/2
print(x,y)
x,y = x/2,y/2
print(x,y)
x,y = x/2,y/2
print(x,y)
x,y = x/2,y/2
print(x,y)


x,y = x*2,y*2
print(x,y)
x,y = x*2,y*2
print(x,y)
x,y = x*2,y*2
print(x,y)
x,y = x*2,y*2
print(x,y)
# x,y = x*3,y*3
# print(x,y)

(x,y),(480, 720), (720, 1080)

In [ ]:
 #720, 1080
240*1.5, 360*1.5

In [ ]:
prepa = [(str(p),'png') for p in pathlib.Path("./prepa").glob('*.png')]

def gen_prepa():
    for img_path, img_type in prepa:
        yield gen_img(img_path, img_type)
        
it = iter(gen_prepa())

img = next(it)
plt.figure()
plt.imshow(img)
plt.figure()
print(img.shape)
y_pred = autoencoder.predict(tf.expand_dims(img, axis=0))
y_pred = tf.cast(y_pred, tf.uint8)
plt.imshow(y_pred[0])


# tf.math.reduce_mean(
#     y_pred, axis=[1,2], keepdims=False, name=None
# )

# y = tf.reshape(y_pred, (1,-1,3))
# y = layers.AveragePooling1D(2)(tf.cast(y, tf.float32))
# y = layers.AveragePooling1D(2)(tf.cast(y, tf.float32))
# y = layers.AveragePooling1D(2)(tf.cast(y, tf.float32))
# # y = tf.image.resize(y, (720, 1080))
# y
y_pred

In [ ]:

initial_learning_rate = 1e-03
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=100,
    decay_rate=0.9,
    staircase=True)

class RenderImages(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        images_path = f"./col_100_output_images/"
        image_name = images_path + f"e_{epoch}.jpg"
#         display.clear_output()
        print_validation(lambda x:autoencoder(x, training=False), batch_size=2, save=False, path="./", datasets=(people_ds, people_test_ds))
        

autoencoder = build_autoencoder()

autoencoder.compile(
                    optimizer="adam",
#                     optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule, beta_1=0.6),
#                 optimizer=tfa.optimizers.AdamW(weight_decay=0.00001, learning_rate=lr_schedule),
#                     loss='mean_squared_error',
                loss=perceptual_loss,
#                     metrics=["accuracy"],
#                    metrics=[LRMetric()],
                )

early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', restore_best_weights=False,
                                                              patience=10, min_delta=0.)  

ghistory = autoencoder.fit(people_ds.prefetch(tf.data.AUTOTUNE),
                                epochs=10, 
#                                     steps_per_epoch=5, 
#                                     steps_per_epoch=small_steps_per_epoch,
                                validation_data=people_test_ds.take(5),
      callbacks=[RenderImages(), early_stop])

In [ ]:
print_validation(lambda x:autoencoder(x, training=False), batch_size=2, save=False, path="./")

In [ ]:
!pip install opencv-python

In [ ]:


for i in range(preds.shape[0]):
    
    plt.figure()
    plt.title("Original")
    plt.imshow(imgs[i])
   
    plt.figure()
    plt.title("OpenCV")
    plt.imshow(cv2.fastNlMeansDenoisingColored(imgs[i].numpy(),None,10,10,7,21))
    plt.figure()
    plt.title("UnsharpMask")
    plt.imshow(unsharp(imgs[i]))
    plt.figure()
    plt.title("UnsharpMask on autoencoder")
    plt.imshow(unsharp(preds[i]))
    plt.title("conservative_smoothing_gray")
    plt.imshow(cv2.bilateralFilter(np.array(unsharp(imgs[i])),9,75,75))
    
    plt.figure()
    plt.title("Autoencoder")
    plt.imshow(preds[i])

In [ ]:
for 